# Mutual funds vs. Index drawdowns

Do active managers reduce drawdowns during downturns?

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()
mfi <- MutualFundsIndia()

2018 was a bad year. So lets look at the subsequent return profile of the top 5 midcap funds entring 2018 through June-2019.

In [ ]:
lastAumDate <- (mfi$AumSchemewise() %>%
    filter(PERIOD <= as.Date('2018-01-01')) %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

lastMetaDate <- (mfi$Meta() %>%
    filter(AS_OF >= lastAumDate) %>%
    summarize(MIN_TS = min(AS_OF)) %>%
    collect())$MIN_TS[1]

print(lastAumDate)
print(lastMetaDate)

top5Aum <- mfi$Meta() %>% 
    inner_join(mfi$AumSchemewise(), by="SCHEME_CODE") %>%
    select(SCHEME_CODE, SCHEME_NAME, AVG_AUM_WO_FOFD) %>%
    filter(PERIOD == lastSwDate & AS_OF == lastMetaDate & CATEGORY == 'Mid-Cap') %>%
    top(n = 5, AVG_AUM_WO_FOFD)

print(top5Aum)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)